# OpenRouter API Key Provisioning for Google Colab

This notebook allows you to bulk provision OpenRouter API keys for multiple users. It's designed to work seamlessly in Google Colab with flexible input options.

## Requirements

- An OpenRouter provisioning API key with admin privileges
- This key can be obtained from your OpenRouter dashboard - https://openrouter.ai/settings/provisioning-keys - you only see this key once when you create it, so store it securely.
- A CSV file with user information (Name, Email columns)

## Security Warning ⚠️⚠️⚠️

⚠️ **This provisioning key is extremely powerful and should be kept secret.**
- Never share this key publicly
- Never commit it to version control
- It can create unlimited API keys and drain your account balance if compromised

## Security on Google Colab

When using Google Colab:
- Your code and data are stored on Google servers
- You can store secret keys in Google Colab's environment variables for better security
- More about Google Secrets: https://guides.library.stanford.edu/api_auth/colab
- DO NOT CODE sensitive information directly in the notebook cells
- Ensure you trust the environment and avoid sharing notebooks with sensitive information
- Always delete sensitive data from the notebook after use - do not leave API keys or user data in the notebook cells! - this notebook does not do it by default. 
- This notebook does not store any data after execution, but be cautious with temporary files.

## Setup and Library Installation

In [ ]:
# Install required libraries
import sys
print(f"Python version: {sys.version}")

# Install dependencies if needed
try:
    import pandas as pd
    print(f"✅ Pandas already installed: {pd.__version__}")
except ImportError:
    print("📦 Installing pandas...")
    !pip install pandas
    import pandas as pd

try:
    from tqdm import tqdm
    print(f"✅ tqdm already installed")
except ImportError:
    print("📦 Installing tqdm...")
    !pip install tqdm
    from tqdm import tqdm

# Standard library imports
import os
import json
import time
import requests
from datetime import datetime
from pathlib import Path
import getpass

print(f"\n✅ All libraries loaded successfully")
print(f"📅 Current date and time: {datetime.now()}")

## Load Provisioning API Key

We'll try to load your OpenRouter provisioning API key from three sources in order:
1. Environment variables
2. Google Colab secrets (if running in Colab)
3. Manual user input (secure, hidden)

### How to set up Google Colab Secrets:
1. Click the 🔑 key icon in the left sidebar
2. Click "+ Add new secret"
3. Name: `OPENROUTER_PROVISIONING_KEY`
4. Value: Your provisioning API key
5. Enable "Notebook access"

In [ ]:
# Attempt 1: Load from environment variables
open_router_provisioning_key = os.getenv("OPENROUTER_PROVISIONING_KEY")

if open_router_provisioning_key:
    print("✅ Provisioning key loaded from environment variables")
else:
    print("⚠️ Provisioning key not found in environment variables")
    
    # Attempt 2: Try Google Colab secrets
    try:
        from google.colab import userdata
        open_router_provisioning_key = userdata.get('OPENROUTER_PROVISIONING_KEY')
        if open_router_provisioning_key:
            print("✅ Provisioning key loaded from Google Colab secrets")
        else:
            raise KeyError("Key not found in secrets")
    except (ImportError, KeyError):
        print("⚠️ Not running in Colab or key not found in Colab secrets")
        
        # Attempt 3: Ask for user input
        print("\n🔑 Please enter your OpenRouter provisioning API key:")
        print("   (Your input will be hidden for security)")
        open_router_provisioning_key = getpass.getpass("Provisioning API Key: ")
        
        # Validate that key was entered
        if not open_router_provisioning_key or open_router_provisioning_key.strip() == "":
            raise ValueError(
                "❌ ERROR: Valid provisioning API key is required to continue.\n"
                "Please obtain a provisioning key from OpenRouter and run this cell again."
            )
        else:
            print("✅ Provisioning key entered successfully")

# Final validation
if open_router_provisioning_key and open_router_provisioning_key.strip():
    print("\n🎉 Provisioning key ready for use")
else:
    raise ValueError("❌ ERROR: No valid provisioning key available")

## Load User Data

Upload or provide a path to a CSV file containing user information.

### CSV Format Requirements:
- Must have at least two columns: `Name` and `Email`
- Optional: `Surname` column for full names
- Example format:

```csv
Name,Email
John Doe,john@example.com
Jane Smith,jane@example.com
```

Or with surname:
```csv
Name,Surname,Email
John,Doe,john@example.com
Jane,Smith,jane@example.com
```

In [ ]:
# Check if running in Google Colab
try:
    from google.colab import files
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

print("📁 Choose how to load your user data:")
print("   1. Upload a CSV file")
print("   2. Provide a file path (e.g., from Google Drive)")

choice = input("\nEnter your choice (1 or 2): ").strip()

df = None

if choice == "1":
    # Upload file
    if IN_COLAB:
        print("\n📤 Please upload your CSV file...")
        uploaded = files.upload()
        
        if uploaded:
            # Get the first uploaded file
            filename = list(uploaded.keys())[0]
            df = pd.read_csv(filename)
            print(f"✅ File '{filename}' uploaded and loaded successfully")
        else:
            print("❌ No file was uploaded")
    else:
        print("⚠️ File upload widget only works in Google Colab")
        print("Please use option 2 to provide a file path instead")
        
elif choice == "2":
    # Provide file path
    file_path = input("\nEnter the full path to your CSV file: ").strip()
    
    if file_path:
        try:
            df = pd.read_csv(file_path)
            print(f"✅ File loaded successfully from: {file_path}")
        except FileNotFoundError:
            print(f"❌ File not found: {file_path}")
        except Exception as e:
            print(f"❌ Error loading file: {e}")
    else:
        print("❌ No file path provided")
else:
    print("❌ Invalid choice. Please run this cell again and enter 1 or 2")

# Validate DataFrame
if df is not None:
    print(f"\n📊 Data loaded: {len(df)} rows, {len(df.columns)} columns")
    
    # Check required columns
    required_columns = ['Name', 'Email']
    missing_columns = [col for col in required_columns if col not in df.columns]
    
    if missing_columns:
        print(f"\n❌ ERROR: Missing required columns: {missing_columns}")
        print(f"Available columns: {list(df.columns)}")
        print("\nPlease ensure your CSV has at least 'Name' and 'Email' columns")
        df = None
    else:
        print("\n✅ Required columns found")
        print("\n📋 Preview of loaded data:")
        display(df.head())
        
        # Check if Surname column exists
        if 'Surname' not in df.columns:
            print("\n⚠️ Note: 'Surname' column not found. Keys will be created using 'Name' and 'Email' only.")
else:
    print("\n❌ No data loaded. Please run this cell again.")

## Configure Key Budget

Set the default credit limit (in USD) for each provisioned API key.

In [ ]:
print("💰 Configure the budget for each API key")
print("\nRecommended limits:")
print("   • Testing/Workshop: $0.50 - $2.00")
print("   • Small projects: $5.00 - $10.00")
print("   • Research projects: $25.00+")

# Get budget from user
budget_input = input("\nEnter budget per key in USD (default: 1.00): ").strip()

if budget_input == "":
    key_budget = 1.00
    print(f"Using default budget: ${key_budget:.2f}")
else:
    try:
        key_budget = float(budget_input)
        if key_budget <= 0:
            print("❌ Budget must be positive. Using default: $1.00")
            key_budget = 1.00
        else:
            print(f"✅ Budget set to: ${key_budget:.2f} per key")
    except ValueError:
        print("❌ Invalid input. Using default budget: $1.00")
        key_budget = 1.00

# Calculate total budget
if df is not None:
    total_budget = key_budget * len(df)
    print(f"\n💵 Total budget for {len(df)} keys: ${total_budget:.2f}")
    
    # Confirmation
    confirm = input("\nProceed with this budget? (yes/no): ").strip().lower()
    if confirm not in ['yes', 'y']:
        print("❌ Budget configuration cancelled. Please run this cell again.")
        key_budget = None
else:
    print("\n⚠️ No user data loaded yet. Load data first before setting budget.")

## Define Key Provisioning Function

In [ ]:
def provision_single_key(provision_key, name, label, limit=1.0):
    """
    Create a single OpenRouter API key using the provisioning API.
    
    Args:
        provision_key: OpenRouter provisioning API key
        name: Name/description for the API key
        label: Label for the API key
        limit: Credit limit in USD (default: 1.0)
    
    Returns:
        dict: API response containing key details, or None if failed
    """
    BASE_URL = "https://openrouter.ai/api/v1/keys"
    
    response = requests.post(
        url=BASE_URL,
        headers={
            "Authorization": f"Bearer {provision_key}",
            "Content-Type": "application/json"
        },
        json={
            "name": name,
            "label": label,
            "limit": limit
        }
    )
    
    if response.status_code == 201:
        return response.json()
    else:
        print(f"❌ Failed to create key for {name}: {response.status_code} - {response.text}")
        return None


def provision_keys_for_users(df, provision_key, key_budget, 
                             name_prefix="API_Key",
                             delay=0.2, 
                             verbose=True):
    """
    Provision OpenRouter API keys for all users in the DataFrame.
    
    Args:
        df: DataFrame with user data (must have 'Name' and 'Email' columns)
        provision_key: OpenRouter provisioning API key
        key_budget: Credit limit per key in USD
        name_prefix: Prefix for key names (default: "API_Key")
        delay: Delay between requests in seconds (default: 0.2)
        verbose: Print detailed progress (default: True)
    
    Returns:
        tuple: (keys_list, updated_df)
    """
    # Validate required columns
    if 'Name' not in df.columns or 'Email' not in df.columns:
        raise ValueError("DataFrame must contain 'Name' and 'Email' columns")
    
    # Initialize
    keys_list = []
    df = df.copy()  # Work with a copy to avoid modifying original
    df['api_key'] = None
    df['key_status'] = None
    
    has_surname = 'Surname' in df.columns
    
    if verbose:
        print(f"\n🚀 Provisioning {len(df)} API keys...")
        print(f"💰 Budget per key: ${key_budget:.2f}")
        print(f"⏱️  Delay between requests: {delay}s\n")
    
    # Iterate through users with progress bar
    for index, row in tqdm(df.iterrows(), total=len(df), desc="Creating keys"):
        # Create unique name and label
        if has_surname:
            full_name = f"{row['Name']}_{row['Surname']}"
        else:
            full_name = row['Name']
        
        name = f"{name_prefix}_{full_name}"
        label = f"{name_prefix}_{row['Email']}"
        
        # Provision the key
        try:
            new_key = provision_single_key(provision_key, name, label, key_budget)
            
            if new_key:
                keys_list.append(new_key)
                df.at[index, 'api_key'] = new_key.get('key', None)
                df.at[index, 'key_status'] = 'success'
            else:
                df.at[index, 'key_status'] = 'failed'
            
            # Rate limiting delay
            time.sleep(delay)
            
        except Exception as e:
            print(f"\n❌ Error provisioning key for {full_name}: {e}")
            df.at[index, 'key_status'] = 'error'
    
    # Summary
    if verbose:
        successful = df['key_status'].value_counts().get('success', 0)
        failed = len(df) - successful
        print(f"\n✅ Provisioning complete:")
        print(f"   • Successful: {successful}/{len(df)}")
        print(f"   • Failed: {failed}/{len(df)}")
        print(f"   • Total budget used: ${successful * key_budget:.2f}")
    
    return keys_list, df

print("✅ Key provisioning functions defined")

## Example: Creating a Single Test Key (Optional)

Uncomment and run this cell to test creating a single key before bulk provisioning.

In [ ]:
# # Test creating a single key
# today = datetime.now().strftime("%Y_%m_%d")
# test_key = provision_single_key(
#     provision_key=open_router_provisioning_key,
#     name=f"Test_Key_{today}",
#     label=f"Test_Key_{today}",
#     limit=0.50  # Small budget for testing
# )

# if test_key:
#     print("✅ Test key created successfully!")
#     print(f"Key ID: {test_key.get('id', 'N/A')}")
#     print(f"Limit: ${test_key.get('limit', 'N/A')}")
#     # Note: We don't print the actual key for security
# else:
#     print("❌ Test key creation failed")

## Provision Keys for All Users

In [ ]:
# Validate prerequisites
if df is None:
    print("❌ ERROR: No user data loaded. Please load a CSV file first.")
elif key_budget is None:
    print("❌ ERROR: Budget not configured. Please set the budget first.")
elif not open_router_provisioning_key:
    print("❌ ERROR: No provisioning key available.")
else:
    # Get name prefix from user
    print("\n🏷️  Configure key naming")
    default_prefix = f"OpenRouter_Key_{datetime.now().strftime('%Y%m%d')}"
    name_prefix = input(f"Enter key name prefix (default: {default_prefix}): ").strip()
    
    if not name_prefix:
        name_prefix = default_prefix
    
    print(f"\nUsing prefix: {name_prefix}")
    print(f"Example key name: {name_prefix}_{df.iloc[0]['Name']}")
    
    # Final confirmation
    print(f"\n⚠️  FINAL CONFIRMATION")
    print(f"   • Users to provision: {len(df)}")
    print(f"   • Budget per key: ${key_budget:.2f}")
    print(f"   • Total budget: ${key_budget * len(df):.2f}")
    print(f"   • Key prefix: {name_prefix}")
    
    confirm = input("\nProceed with provisioning? (yes/no): ").strip().lower()
    
    if confirm in ['yes', 'y']:
        # Provision keys
        keys_list, updated_df = provision_keys_for_users(
            df=df,
            provision_key=open_router_provisioning_key,
            key_budget=key_budget,
            name_prefix=name_prefix,
            delay=0.2,
            verbose=True
        )
        
        # Update the global dataframe
        df = updated_df
        
        print("\n📊 Provisioning Results:")
        display(df[['Name', 'Email', 'key_status']].head(10))
        
        if len(df) > 10:
            print(f"\n... and {len(df) - 10} more rows")
        
    else:
        print("❌ Provisioning cancelled")

## Save Results

Save the provisioned keys and updated user data to files.

In [ ]:
if df is not None and 'api_key' in df.columns:
    # Create output directory
    output_dir = Path("provisioned_keys")
    output_dir.mkdir(exist_ok=True)
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Save DataFrame as CSV
    csv_path = output_dir / f"provisioned_keys_{timestamp}.csv"
    df.to_csv(csv_path, index=False)
    print(f"✅ User data with API keys saved to: {csv_path}")
    
    # Save keys list as JSON
    if 'keys_list' in locals() and keys_list:
        json_path = output_dir / f"keys_details_{timestamp}.json"
        with open(json_path, 'w', encoding='utf-8') as f:
            json.dump(keys_list, f, indent=2, ensure_ascii=False)
        print(f"✅ Key details saved to: {json_path}")
    
    # Create a summary report
    summary_path = output_dir / f"summary_{timestamp}.txt"
    with open(summary_path, 'w', encoding='utf-8') as f:
        f.write(f"OpenRouter API Key Provisioning Summary\n")
        f.write(f"="*50 + "\n\n")
        f.write(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Total users: {len(df)}\n")
        f.write(f"Successful: {df['key_status'].value_counts().get('success', 0)}\n")
        f.write(f"Failed: {len(df) - df['key_status'].value_counts().get('success', 0)}\n")
        f.write(f"Budget per key: ${key_budget:.2f}\n")
        f.write(f"Total budget: ${key_budget * df['key_status'].value_counts().get('success', 0):.2f}\n")
    
    print(f"✅ Summary report saved to: {summary_path}")
    
    # Download files if in Colab
    if IN_COLAB:
        print("\n📥 Download files?")
        download = input("Download all files to your computer? (yes/no): ").strip().lower()
        if download in ['yes', 'y']:
            files.download(str(csv_path))
            if 'keys_list' in locals() and keys_list:
                files.download(str(json_path))
            files.download(str(summary_path))
            print("✅ Files downloaded")
    
    print("\n⚠️  IMPORTANT SECURITY REMINDERS:")
    print("   • Keep these files secure and private")
    print("   • Do not share API keys publicly")
    print("   • Do not commit these files to public repositories")
    print("   • Distribute keys securely to intended recipients")
    
else:
    print("❌ No keys to save. Please provision keys first.")

## View Provisioning Statistics

In [ ]:
if df is not None and 'key_status' in df.columns:
    print("📊 PROVISIONING STATISTICS\n")
    print("="*50)
    
    status_counts = df['key_status'].value_counts()
    
    for status, count in status_counts.items():
        percentage = (count / len(df)) * 100
        print(f"{status.upper():10}: {count:3} ({percentage:5.1f}%)")
    
    print("="*50)
    
    successful = status_counts.get('success', 0)
    total_spent = successful * key_budget
    
    print(f"\n💰 Financial Summary:")
    print(f"   Budget per key: ${key_budget:.2f}")
    print(f"   Keys created: {successful}")
    print(f"   Total spent: ${total_spent:.2f}")
    
    # Show failed keys if any
    failed_df = df[df['key_status'] != 'success']
    if len(failed_df) > 0:
        print(f"\n⚠️  Failed Keys ({len(failed_df)}):")
        display(failed_df[['Name', 'Email', 'key_status']])
else:
    print("❌ No provisioning data available")

## Next Steps

After provisioning keys:

1. **Secure Distribution**: Send API keys to users through secure channels (encrypted email, secure messaging)
2. **Usage Instructions**: Provide users with documentation on how to use their keys
3. **Monitor Usage**: Regularly check OpenRouter dashboard for usage statistics
4. **Key Management**: Keep track of which keys belong to which users
5. **Revocation Plan**: Have a process to revoke keys if needed

### Security Best Practices:

- ✅ Never share keys in plain text
- ✅ Use encrypted communication for distribution
- ✅ Keep the provisioning key extremely secure
- ✅ Regularly audit key usage
- ✅ Set appropriate budget limits
- ✅ Have a key rotation policy